In [ ]:
import harpy as hp

If `sopa` not yet installed, run the following in an active conda environment:

`pip install sopa`

In [ ]:
from datasets import sdata_xenium

sdata=sdata_xenium(path = None,  output=None ) # sdata_resolve( output=None )  # on Windows, set path (e.g. to r"c:\tmp")
sdata

In [ ]:
img_layer = "clahe"
labels_layer = "segmentation_mask"
table_layer = "table_transcriptomics_clustered"
points_layer = "transcripts_global" # "transcripts"

In [ ]:
from harpy.utils._keys import _REGION_KEY, _INSTANCE_KEY

assert labels_layer in sdata[ table_layer ].obs[ _REGION_KEY ].cat.categories.to_list()

In [ ]:
from spatialdata.transformations import get_transformation

transformations = get_transformation( sdata[ labels_layer ], get_all=True )
# vectorize the labels layer that annotates the table
hp.sh.vectorize( sdata, labels_layer=labels_layer, output_layer="__shapes__", overwrite=True )

sdata[ "__shapes__" ]=sdata[ "__shapes__" ].copy()

# subset with the region key first
adata = sdata[ table_layer ][ sdata[ table_layer ].obs[ _REGION_KEY ] == labels_layer]
assert all( adata.obs[ _INSTANCE_KEY ].isin( sdata[ "__shapes__" ].index.astype( int ))), "There are elements in table layer that could not be found in shapes layer."
sdata[ "__shapes__" ].index=sdata[ "__shapes__" ].index.astype( int )
# subset the shapes layer with the instance key, and make sure they are in the same order
sdata[ "__shapes__" ]=sdata[ "__shapes__" ].loc[ adata.obs[ _INSTANCE_KEY ].values ]

In [ ]:
from harpy.image.segmentation.segmentation_models._baysor import _ensure_polygon
from shapely.validation import make_valid

def _make_valid(gdf):
    gdf.geometry = gdf.geometry.map(lambda cell: _ensure_polygon(make_valid(cell)))
    assert all( ~gdf.geometry.isna() ) # check that all polygons are valid
    return gdf

sdata[ "__shapes__" ]=_make_valid( sdata["__shapes__" ] )
assert sdata[ "__shapes__" ].shape[0] == adata.shape[0]  # should have same shape, and have same order

In [ ]:
import os
import uuid
import sopa
import tempfile

from harpy.utils._keys import _GENES_KEY

OUTPUT_DIR =  tempfile.gettempdir()

xenium_explorer_output_path = os.path.join( OUTPUT_DIR, f"xenium_{uuid.uuid4()}.explorer" )

sopa.io.write(
    xenium_explorer_output_path,
    sdata,
    image_key=img_layer,
    points_key=points_layer,
    shapes_key="__shapes__" ,
    gene_column=_GENES_KEY,
    pixel_size = 0.2125,
    )

In [ ]:
import numpy as np

adata.obs['new_column'] = np.random.randint(0, 12, size=len(adata.obs))
adata.obs[ 'new_column' ]=adata.obs[ 'new_column' ].astype( "category" )

In [ ]:
sopa.io.write_cell_categories( xenium_explorer_output_path, adata, is_dir=True )

In [ ]:
import scanpy as sc

sc.pl.umap(sdata.tables[table_layer], color=["leiden"], show=True)
hp.pl.plot_shapes(
    sdata,
    img_layer="clahe",
    table_layer=table_layer,
    column="leiden",
    shapes_layer="__shapes__",
    alpha=1,
    linewidth=0,
    channel="DAPI",
)

In [ ]:
import scanpy as sc

sdata.tables[ table_layer ].obs[ "new_column" ] = adata.obs[ "new_column" ]

sc.pl.umap(sdata.tables[table_layer], color=["new_column"], show=True)
hp.pl.plot_shapes(
    sdata,
    img_layer="clahe",
    table_layer=table_layer,
    column="new_column",
    shapes_layer="__shapes__",
    alpha=1,
    linewidth=0,
    channel="DAPI",
)

In [ ]:
sdata.tables[ table_layer ].obs[ sdata.tables[ table_layer ].obs[ "cell_ID" ] == 312]

In [ ]:
from napari_spatialdata import Interactive

# for new version of napari-spatialdata, need to remove the colors first
#del sdata[ table_layer ].uns[ "new_column_colors" ]
#Interactive( sdata )